人类呼吸频率0.05-0.5hz
采样12个frame时间：大约6.5s
6.5秒内呼吸次数在0.32-3.2次之间



In [5]:

import numpy as np
import os

In [10]:
import numpy as np

def generate_random_masks(height, width, window_size=8, num_windows=4, num_masks=5):
    """
    生成指定维度的随机掩码。

    参数:
        height (int): 每个掩码的高度。
        width (int): 每个掩码的宽度。
        window_size (int): 含有零行的每个窗口的大小。
        num_windows (int): 每个掩码中的窗口数量。
        num_masks (int): 要生成的掩码数量。

    返回:
        numpy.ndarray: 形状为 (num_masks, height, width) 的数组，包含生成的掩码。
    """
    masks_list = []

    for _ in range(num_masks):
        mask = np.ones((height, width))  # 初始化所有元素为1的掩码
        available_height = height - (window_size * num_windows)  # 计算可用的高度，以确保窗口不会超出边界
        
        if available_height < 0:
            raise ValueError("指定的窗口数和大小超出了掩码的高度范围")
        
        # 随机生成每个窗口的起始位置
        window_starts = np.random.choice(available_height + 1, num_windows, replace=False)
        
        # 对窗口起始位置进行排序
        window_starts.sort()
        
        # 在每个窗口的起始位置处设置零行
        for start_pos in window_starts:
            mask[start_pos:start_pos + window_size, :] = 0
        
        masks_list.append(mask)

    return np.array(masks_list)



In [18]:
def save_mask(load_root,save_root,window_size,num_window,mask_num):
    patient_list=os.listdir(load_root)
    save_root=save_root+f"_{mask_num}x{num_window}x{window_size}"
    if not os.path.exists(save_root):
        os.mkdir(save_root)
    for patient in sorted(patient_list):
        load_patient_path=os.path.join(load_root,patient)
        save_patient_path=os.path.join(save_root,patient)
        if not os.path.exists(save_patient_path):
            os.mkdir(save_patient_path)
        file_list=os.listdir(load_patient_path)
        for file in file_list:
            if 'msak' in file:
                continue
            else:
                file_name=os.path.splitext(file)[0]
                file_path=os.path.join(load_patient_path,file)
                mask=generate_random_masks(128,128,window_size=window_size,num_windows=num_window,num_masks=mask_num).astype(np.float32)
                
                file_save_name=file_name+f'_mask.npy'
                save_file_path=os.path.join(save_patient_path,file_save_name)
                np.save(save_file_path,mask)
                    

In [19]:
load_root="/mnt/datasets/CMR/MICCAIChallenge2023/ChallengeData/SingleCoil/Cine/PD/train/TrainingSet/FullSample_slice"
save_root="/mnt/datasets/CMR/MICCAIChallenge2023/ChallengeData/SingleCoil/Cine/PD/train/TrainingSet/Noise_Mask"

In [20]:
save_mask(load_root,save_root,4,8,6)